In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import torch
from botorch.models import SingleTaskGP, ModelListGP, fully_bayesian
from botorch.models.gp_regression_mixed import MixedSingleTaskGP
from botorch.fit import fit_gpytorch_mll, fit_fully_bayesian_model_nuts
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement, qUpperConfidenceBound
from botorch.acquisition.analytic import UpperConfidenceBound, ProbabilityOfImprovement, ExpectedImprovement
from botorch.optim import optimize_acqf, optimize_acqf_mixed
from botorch.cross_validation import gen_loo_cv_folds
import math
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, DotProduct, RationalQuadratic, WhiteKernel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [5]:
train_X = torch.cat([torch.rand(20, 2), torch.randint(3, (20, 1))], dim=-1)
train_X

tensor([[0.6399, 0.8919, 1.0000],
        [0.1974, 0.9955, 0.0000],
        [0.3863, 0.4437, 1.0000],
        [0.6449, 0.4248, 1.0000],
        [0.3050, 0.1107, 1.0000],
        [0.0477, 0.1151, 2.0000],
        [0.3416, 0.7841, 0.0000],
        [0.6536, 0.0485, 2.0000],
        [0.1016, 0.5855, 0.0000],
        [0.6309, 0.0042, 0.0000],
        [0.0772, 0.2236, 0.0000],
        [0.7975, 0.9260, 1.0000],
        [0.2833, 0.3631, 1.0000],
        [0.2112, 0.9375, 1.0000],
        [0.3579, 0.2989, 1.0000],
        [0.7984, 0.6408, 2.0000],
        [0.2473, 0.6191, 1.0000],
        [0.8345, 0.2529, 0.0000],
        [0.9881, 0.1619, 2.0000],
        [0.3804, 0.6216, 0.0000]])

In [6]:
train_Y = (
        torch.sin(train_X[..., :-1]).sum(dim=1, keepdim=True)
        + train_X[..., -1:]
    )
train_Y

tensor([[2.3754],
        [1.0351],
        [1.8061],
        [2.0133],
        [1.4107],
        [2.1625],
        [1.0412],
        [2.6565],
        [0.6541],
        [0.5941],
        [0.2989],
        [2.5148],
        [1.6347],
        [2.0157],
        [1.6448],
        [3.3141],
        [1.8251],
        [0.9912],
        [2.9962],
        [0.9537]])

In [7]:
model = MixedSingleTaskGP(train_X, train_Y, cat_dims=[-1])

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/botorch/models/gpytorch.py:113: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(


In [8]:
EI = qExpectedImprovement(
    model = model,
    best_f = max(train_Y),
)

In [15]:
X_candidates,_ = optimize_acqf_mixed(
    acq_function = EI,
    bounds = torch.tensor([[0., 0., 0.,],[1., 1., 1.]]),
    q = 5, # Number of suggested candidates, 20 candidates took around 40 mins
    num_restarts = 200,
    raw_samples = 512,
    constraints = [(torch.tensor([1, 2, 3, 4]), torch.tensor([1.]*4), 1.0)],
    fixed_features_list = None
)
X_candidates

ValueError: fixed_features_list must be non-empty.